<a href="https://colab.research.google.com/github/Byung-seok/code_snippet_/blob/main/spring_wandb_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from google.colab import files
from google.colab import drive

In [10]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name} with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

In [2]:
# dataset_path = 'C:/Users/HDS/Documents/Python/Work/2021_data/data/220408_워터박스 데이터셋_17년-22년3월.csv'
dataset_path = './220408_워터박스 데이터셋_17년-22년3월.csv'
raw_dataset = pd.read_csv(dataset_path, na_values = "", comment='\t', sep=",", skipinitialspace=True, encoding = 'cp949')

raw_dataset_nan_drop = raw_dataset.dropna()
raw_dataset_nan_drop.reset_index(inplace = True, drop = True)

set_zero = set()
zero_allowance_include_date = ['', '', 'N', '', '', '', 'N', 'N', 'Y', 'N', 'N', '', '', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N']

for ind, key in enumerate(raw_dataset_nan_drop.keys()):
    if zero_allowance_include_date[ind] == 'N':
        set_zero.update(raw_dataset_nan_drop[raw_dataset_nan_drop[key] == 0].index.tolist())

raw_dataset_nan_zero_drop = raw_dataset_nan_drop.drop(set_zero)
raw_dataset_nan_zero_drop.reset_index(inplace = True, drop = True)

grade_group = []

for i in range(len(raw_dataset_nan_zero_drop)):
    if raw_dataset_nan_zero_drop['규격약호'][i] in ["SCM435W", "SCM435W-J", "SCM435HW", "SCM435W-EX", "SCM435HW-J", "SCM435HW-M1", "SCM415HW", "SCM440W", "SCM440HW"]:
        grade_group.append(1)
    elif raw_dataset_nan_zero_drop['규격약호'][i] in ['SAE9254W-SI', 'SAE9254W-M2', 'SAE9254W-M1']:
        grade_group.append(2)
    elif raw_dataset_nan_zero_drop['규격약호'][i] in ['SWRCH10A', 'SWRCH45K', 'SWRCH18A-M1', 'SWRCH10A-M1', 'SWRCH18A', 'SWRCH25K', 'SWRCH35K', 'SWRCH45CR', 'SWRCH45KS', 'SWRCH35KS']:
        grade_group.append(3)
    elif raw_dataset_nan_zero_drop['규격약호'][i] in ['S1021B']:
        grade_group.append(4)
    elif raw_dataset_nan_zero_drop['규격약호'][i] in ['SAE1010', 'SAE1010-M1', 'SAE1018']:
        grade_group.append(5)
    elif raw_dataset_nan_zero_drop['규격약호'][i] in ['ER70S-T', 'ER70S-G']:
        grade_group.append(6)
    else:
        grade_group.append(0)
        
df = raw_dataset_nan_zero_drop.copy()
df['규격약호'] = grade_group

df = df[(df['규격약호'] != 0)]
df = df[(df['규격약호'] != 1)]
# df = df[(df['규격약호'] != 2)]
df = df[(df['규격약호'] != 3)]
df = df[(df['규격약호'] != 4)]
df = df[(df['규격약호'] != 5)]
df = df[(df['규격약호'] != 6)]

df.reset_index(inplace = True, drop = True)
test_start_idx = df[(df['압연계상연도'] == 2022) & ((df['압연계상월'] == 3) | (df['압연계상월'] == 4))].index[0]
# test_start_idx = df[(df['압연계상연도'] == 2022) & ((df['압연계상월'] == 1) | (df['압연계상월'] == 2))].index[0] # 신업재 3,4월 실적 없음

df.drop(['RM입측소재온도','IM2출측소재온도','PFM#2출측소재온도','NTM출측온도','RSM입측온도','권취온도','NTM SPEED','RSM SPEED','Stand23사용량','Stand24사용량',
              'Stand25사용량','Stand26사용량','Stand27사용량','Stand28사용량','Stand29사용량','Stand30사용량','Stand31사용량','Stand32사용량','Stand33사용량',
              'Stand34사용량','PFM후단면적','NTM후단면적','RSM후단면적','RM R/T 커버','IM R/T 커버','압연계상연도','압연계상월','압연계상일','규격약호',
              #'Zone1WZ1압력','1-1 2차밸브','Zone1WZ2유량','Zone1WZ2압력','1-2 2차밸브',
              'Zone2WZ1유량','Zone2WZ1압력','2-1 2차밸브','Zone2WZ2유량','Zone2WZ2압력','2-2 2차밸브',
              'Zone2WZ3유량','Zone2WZ3압력','2-3 2차밸브','Zone2WZ4유량','Zone2WZ4압력','2-4 2차밸브','Zone2 공급압력',
              'Zone3WZ1유량','Zone3WZ1압력','3-1 2차밸브','Zone3WZ2유량','Zone3WZ2압력','3-2 2차밸브','Zone3 공급압력'
            ,'STAND 1 SPEED','STAND 2 SPEED','STAND 3 SPEED','STAND 4 SPEED','STAND 5 SPEED','STAND 6 SPEED','STAND 7 SPEED','STAND 8 SPEED'
            ,'STAND 9 SPEED','STAND 10 SPEED','STAND 11 SPEED','STAND 12 SPEED','STAND 13 SPEED','STAND 14 SPEED','STAND 15 SPEED'
            ,'STAND 16 SPEED','STAND 17 SPEED','STAND 18 SPEED'
            ,'Stand1사용량','Stand2사용량','Stand3사용량','Stand4사용량','Stand5사용량','Stand6사용량','Stand7사용량','Stand8사용량','Stand9사용량'
            ,'Stand10사용량','Stand11사용량','Stand12사용량','Stand13사용량','Stand14사용량','Stand15사용량','Stand16사용량','Stand17사용량','Stand18사용량'
            ,'Stand19사용량','Stand20사용량','Stand21사용량','Stand22사용량','RM후단면적','STD18후단면적'
            ,'PFM1 SPEED', 'PFM2 SPEED'
             ],axis=1,inplace=True)

target = df['NTM입측온도']
dataset = df.drop(['NTM입측온도'], axis = 1)

train_dataset = dataset.iloc[:test_start_idx]
train_target = target.iloc[:test_start_idx]
test_dataset = dataset.iloc[test_start_idx:]
test_target = target.iloc[test_start_idx:]

train_stats = train_dataset.describe()
train_stats = train_stats.transpose()

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

train_norm = norm(train_dataset)
test_norm = norm(test_dataset)

num_epochs = 100
size_batch = 256
rate_learning = 1e-2

wandb.init(project = 'spring_sweep_bayes',
           entity = 'jbee22',
           config = {
               'num_epochs': num_epochs,
               'rate_learning': rate_learning,
               'size_batch': size_batch
           })
config = wandb.config

def build_network(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(128, activation = 'relu', input_shape = input_shape))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dense(32, activation = 'relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=config.rate_learning), loss = 'mse', metrics = [tf.keras.metrics.RootMeanSquaredError()])
    return model

model = build_network((train_norm.shape[1],))
model.summary()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
wandb: Currently logged in as: jbee22 (use `wandb login --relogin` to force relogin)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1920      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.fit(train_norm, train_target, epochs = config.num_epochs, batch_size = config.size_batch, validation_data = (test_norm, test_target), callbacks = [WandbCallback()])
wandb.finish()

Epoch 1/100
41/41 [==============================] - 1s 9ms/step - loss: 413950.4062 - root_mean_squared_error: 643.3898 - val_loss: 63912.0312 - val_root_mean_squared_error: 252.8083 - _timestamp: 1650872243.0000 - _runtime: 6.0000
Epoch 2/100
41/41 [==============================] - 0s 4ms/step - loss: 23141.3594 - root_mean_squared_error: 152.1228 - val_loss: 21780.5625 - val_root_mean_squared_error: 147.5824 - _timestamp: 1650872243.0000 - _runtime: 6.0000
Epoch 3/100
41/41 [==============================] - 0s 4ms/step - loss: 9139.0625 - root_mean_squared_error: 95.5984 - val_loss: 12659.6602 - val_root_mean_squared_error: 112.5152 - _timestamp: 1650872243.0000 - _runtime: 6.0000
Epoch 4/100
41/41 [==============================] - 0s 4ms/step - loss: 5522.4863 - root_mean_squared_error: 74.3134 - val_loss: 8942.3506 - val_root_mean_squared_error: 94.5640 - _timestamp: 1650872243.0000 - _runtime: 6.0000
Epoch 5/100
41/41 [==============================] - 0s 5ms/step - loss: 3650

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,99
best_val_loss,266.43741
epoch,99
loss,79.7602
root_mean_squared_error,8.93086
val_loss,266.43741


In [4]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'num_epochs': {
            'values': [10, 50, 100, 150, 200]
        },
        'size_batch': {
            'values': [8, 16, 32, 64, 128, 256, 512]
        },
        'rate_learning': {
            'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = 'jbee22', project = 'spring_sweep_bayes')

def train():
  config_defaults = {
      'num_epochs' : 100,
      'size_batch' : 256,
      'rate_learning' : 0.01,
  }

  wandb.init(config = config_defaults)

  config = wandb.config

  model = build_network((train_norm.shape[1],))
  model.fit(train_norm, train_target, epochs = config.num_epochs, batch_size = config.size_batch
            , validation_data = (test_norm, test_target), callbacks = [wandb.keras.WandbCallback()])

wandb.agent(sweep_id, train, count = 100)

Create sweep with ID: jb0fdiya
Sweep URL: https://wandb.ai/jbee22/spring_sweep_bayes/sweeps/jb0fdiya


wandb: Agent Starting Run: ck84svpk with config:
wandb: 	num_epochs: 50
wandb: 	rate_learning: 0.0005
wandb: 	size_batch: 256


Epoch 1/50
41/41 [==============================] - 1s 8ms/step - loss: 408816.6562 - root_mean_squared_error: 639.3877 - val_loss: 44145.5430 - val_root_mean_squared_error: 210.1084 - _timestamp: 1650872273.0000 - _runtime: 5.0000
Epoch 2/50
41/41 [==============================] - 0s 5ms/step - loss: 21322.5469 - root_mean_squared_error: 146.0224 - val_loss: 15726.6875 - val_root_mean_squared_error: 125.4061 - _timestamp: 1650872273.0000 - _runtime: 5.0000
Epoch 3/50
41/41 [==============================] - 0s 5ms/step - loss: 9160.4736 - root_mean_squared_error: 95.7104 - val_loss: 11894.7021 - val_root_mean_squared_error: 109.0628 - _timestamp: 1650872274.0000 - _runtime: 6.0000
Epoch 4/50
41/41 [==============================] - 0s 5ms/step - loss: 5531.8477 - root_mean_squared_error: 74.3764 - val_loss: 10083.3330 - val_root_mean_squared_error: 100.4158 - _timestamp: 1650872274.0000 - _runtime: 6.0000
Epoch 5/50
41/41 [==============================] - 0s 4ms/step - loss: 3892.66

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,48
best_val_loss,719.341
epoch,49
loss,121.17133
root_mean_squared_error,11.00778
val_loss,744.07471


wandb: Agent Starting Run: yyw13bm1 with config:
wandb: 	num_epochs: 50
wandb: 	rate_learning: 0.0005
wandb: 	size_batch: 32


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: xwzu971g with config:
wandb: 	num_epochs: 200
wandb: 	rate_learning: 0.005
wandb: 	size_batch: 128


Epoch 1/200
82/82 [==============================] - 1s 6ms/step - loss: 206481.7188 - root_mean_squared_error: 454.4026 - val_loss: 17879.2480 - val_root_mean_squared_error: 133.7133 - _timestamp: 1650873124.0000 - _runtime: 7.0000
Epoch 2/200
82/82 [==============================] - 0s 3ms/step - loss: 8249.4492 - root_mean_squared_error: 90.8265 - val_loss: 9834.1484 - val_root_mean_squared_error: 99.1673 - _timestamp: 1650873124.0000 - _runtime: 7.0000
Epoch 3/200
82/82 [==============================] - 0s 4ms/step - loss: 3579.2390 - root_mean_squared_error: 59.8267 - val_loss: 7806.6738 - val_root_mean_squared_error: 88.3554 - _timestamp: 1650873124.0000 - _runtime: 7.0000
Epoch 4/200
82/82 [==============================] - 0s 4ms/step - loss: 2059.8152 - root_mean_squared_error: 45.3852 - val_loss: 5560.1084 - val_root_mean_squared_error: 74.5661 - _timestamp: 1650873125.0000 - _runtime: 8.0000
Epoch 5/200
82/82 [==============================] - 0s 4ms/step - loss: 1442.2437 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▃▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▅▄▄▃▂▂▂▂▂▁▁▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,198
best_val_loss,93.24194
epoch,199
loss,64.04669
root_mean_squared_error,8.00292
val_loss,191.95317


wandb: Agent Starting Run: t5hsmab7 with config:
wandb: 	num_epochs: 150
wandb: 	rate_learning: 0.005
wandb: 	size_batch: 128


Epoch 1/150
82/82 [==============================] - 1s 6ms/step - loss: 218475.1875 - root_mean_squared_error: 467.4133 - val_loss: 21209.5234 - val_root_mean_squared_error: 145.6349 - _timestamp: 1650873201.0000 - _runtime: 7.0000
Epoch 2/150
82/82 [==============================] - 0s 4ms/step - loss: 9015.6396 - root_mean_squared_error: 94.9507 - val_loss: 11481.1602 - val_root_mean_squared_error: 107.1502 - _timestamp: 1650873201.0000 - _runtime: 7.0000
Epoch 3/150
82/82 [==============================] - 0s 4ms/step - loss: 4097.8716 - root_mean_squared_error: 64.0146 - val_loss: 6480.2417 - val_root_mean_squared_error: 80.4999 - _timestamp: 1650873202.0000 - _runtime: 8.0000
Epoch 4/150
82/82 [==============================] - 0s 4ms/step - loss: 2432.4526 - root_mean_squared_error: 49.3199 - val_loss: 5507.2090 - val_root_mean_squared_error: 74.2106 - _timestamp: 1650873202.0000 - _runtime: 8.0000
Epoch 5/150
82/82 [==============================] - 0s 4ms/step - loss: 1626.489